In [23]:
# configuration
config = {
    'FILE_PATH': '/Users/pradeep/Desktop/ProjectANotebooks/notebooks/processed_dataset.csv'
    }
config

{'FILE_PATH': '/Users/pradeep/Desktop/ProjectANotebooks/notebooks/processed_dataset.csv'}

In [24]:
# import libraries
try:
    import pandas as pd
    import pickle
    import numpy as np
    import matplotlib.pyplot as plt
    import string
    import re
    from time import time
    from imblearn.over_sampling import SMOTE
    import texthero as hero
    from texthero import preprocessing
    from gensim.models import Word2Vec
    from sklearn.model_selection import cross_val_score
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import StackingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    import warnings
except(ImportError):
    print(f'Import Error: {ImportError}')

# ignore warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# set seeds for reproducability
from numpy.random import seed
seed(500)

# global configurations
pd.set_option("display.max_colwidth", -1) # show larger text in pandas dataframe

In [25]:
'''
- select features
- drop missing values
- combine selected features into one
'''
def data(df):
    #new_df = df[['EventDescription', 'FailedAssets', 'IncidentCause', 'IncidentConsequence', 'IncidentType', 'Status', 'WeatherStation', 'Category']]
    #new_df.dropna(axis=0, inplace=True)
    features = df['EventDescription'] +' ' + df['Address'] + ' '+ df['IncidentCause'] +' ' + df['ActionTaken'] +\
    ' '+ df['FailedAssets'] +' ' + df['Locality']  + ' ' + df['IncidentConsequence'] + ' ' + \
    df['CauseTechnical'] + ' ' + df['CauseTechnical']
    target = df['Category']
    return features, target

In [26]:
# read csv
dataset = pd.read_csv(config['FILE_PATH'])

# get features and target
features, target = data(dataset)

# Text Cleaning and Pre-processing
def preprocess_text():
    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
        #preprocessing.tokenize
    ]

    # apply pipeline to text
    clean_text = features.pipe(hero.clean, cleaning_pipeline)

    return clean_text

In [27]:
# check processed text
clean_text = preprocess_text()
clean_text

KeyboardInterrupt: 

In [ ]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(clean_text, target, random_state=0, test_size=0.25, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
# feature extraction methods
# tfidf
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=1000)
    vectorizer.fit(clean_text)
    with open('TFIDF_1000.pickle', 'wb') as f:
        pickle.dump(vectorizer, f)
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

# bow
def bow():
    count_vectorizer = CountVectorizer(analyzer='word', max_features=1000)
    count_vectorizer.fit(clean_text)
    train_bow = count_vectorizer.transform(x_train)
    test_bow = count_vectorizer.transform(x_test)
    return train_bow, test_bow

# bigrams
def bigrams():
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2), max_features=1000)
    bigram_vectorizer.fit(clean_text)
    train_bigram = bigram_vectorizer.transform(x_train)
    test_bigram = bigram_vectorizer.transform(x_test)
    return train_bigram, test_bigram

In [ ]:
# get features
train_tfidf, test_tfidf = tfidf()
train_tfidf.shape, test_tfidf.shape

train_bigram, test_bigram = bigrams()

train_bow, test_bow = bow()

In [ ]:
oversample = SMOTE(random_state=0,n_jobs=-1,k_neighbors=5)
train_tfidf, y_train = oversample.fit_resample(train_tfidf, y_train)
print(f'Shape: {train_tfidf.shape}')
print(y_train.value_counts())

In [ ]:
# random forest classifier with TFIDF
rf = RandomForestClassifier(random_state=0)

# fit
rf.fit(train_tfidf, y_train)

# predict
y_pred = rf.predict(test_tfidf)

# accuracy
accuracy_score(y_test, y_pred)

In [ ]:
# grid search
param_grid = {
    'n_estimators': [380, 400, 420, 450, 500],
    #'max_depth' : [20,30,35,40,45],
    'criterion' :['gini']
}
CV_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=0),
                      param_grid=param_grid, cv= 5, n_jobs=-1)
CV_rfc.fit(train_tfidf, y_train)
print(CV_rfc.best_params_)
print(CV_rfc.best_score_)

In [28]:
# random forest classifier
rf = RandomForestClassifier(random_state=0, criterion='gini',  n_estimators=390)

# fit
rf.fit(train_tfidf, y_train)

#with open('random_forest.pickle', 'wb') as f:
   # pickle.dump(rf, f)

# predict
y_pred = rf.predict(test_tfidf)

# accuracy
print(accuracy_score(y_test, y_pred))

# classification report
print(classification_report(y_test, y_pred))


0.8175046554934823
              precision    recall  f1-score   support

    AF Other       0.56      0.52      0.54        93
      Animal       0.86      0.85      0.86        67
   Conductor       0.51      0.68      0.58        34
  Connection       0.67      0.82      0.74       210
    Crossarm       0.85      0.92      0.88       130
      Dug up       0.89      0.92      0.91        92
        Fuse       0.86      0.80      0.83       107
Installation       0.00      0.00      0.00         8
   Lightning       0.97      0.78      0.86        41
    OH Cable       0.71      0.65      0.68        60
       Other       0.95      0.85      0.90       325
        Pole       0.67      0.54      0.60        83
       Trees       0.88      0.95      0.91       134
    UG Cable       0.00      0.00      0.00         6
     Vehicle       0.89      0.94      0.91       221

    accuracy                           0.82      1611
   macro avg       0.69      0.68      0.68      1611
weighte